In [98]:
import pandas as pd

df_origin = pd.read_csv("HK_REEXPORT_ALL_ORIGIN_2023_MAR.csv")
df_consign = pd.read_csv("HK_REEXPORT_ALL_CONSIGNMENT_2023_MAR.csv")
df_iso = pd.read_csv("HK_ISO_index.csv")
df_rate = pd.read_csv("HKD_USD_RATE.csv")

# remove all notes at the bottom of the table
def remove_notes_at_footer(df):
    eol = 0
    for index, value in df.iterrows():
        if str(df.loc[index, "Year"]).isnumeric():
            eol = index
    df = df.loc[:eol, :]
    return df

def format_date(df):
     for index, value in df.iterrows():
          if df.loc[index, "Month"] == 'Jan':
               df.loc[index, "Year"] = str(df.loc[index, "Year"]) + "-01-01"
          elif df.loc[index, "Month"] == 'Feb':
               df.loc[index, "Year"] = str(df.loc[index, "Year"]) + "-02-01"
          elif df.loc[index, "Month"] == 'Mar':
               df.loc[index, "Year"] = str(df.loc[index, "Year"]) + "-03-01"
          elif df.loc[index, "Month"] == 'Apr':
               df.loc[index, "Year"] = str(df.loc[index, "Year"]) + "-04-01"
          elif df.loc[index, "Month"] == 'May':
               df.loc[index, "Year"] = str(df.loc[index, "Year"]) + "-05-01"
          elif df.loc[index, "Month"] == 'Jun':
               df.loc[index, "Year"] = str(df.loc[index, "Year"]) + "-06-01"
          elif df.loc[index, "Month"] == 'Jul':
               df.loc[index, "Year"] = str(df.loc[index, "Year"]) + "-07-01"
          elif df.loc[index, "Month"] == 'Aug':
               df.loc[index, "Year"] = str(df.loc[index, "Year"]) + "-08-01"
          elif df.loc[index, "Month"] == 'Sep':
               df.loc[index, "Year"] = str(df.loc[index, "Year"]) + "-09-01"
          elif df.loc[index, "Month"] == 'Oct':
               df.loc[index, "Year"] = str(df.loc[index, "Year"]) + "-10-01"
          elif df.loc[index, "Month"] == 'Nov':
               df.loc[index, "Year"] = str(df.loc[index, "Year"]) + "-11-01"
          elif df.loc[index, "Month"] == 'Dec':
               df.loc[index, "Year"] = str(df.loc[index, "Year"]) + "-12-01"
          
     # change the column name to 'Date' after concatenation
     df.rename(columns={"Year" : "DATE"}, inplace=True)
     df.drop(columns=["Month"], axis=0, inplace=True)
     return df

# add ISO code to dataframe using left join on country code column
def update_iso(df,df_iso, country_col):
# add country for import file iso code from iso csv file 
    df = df.merge(df_iso[['origin', 'ISO']], left_on=country_col, right_on='origin', how='left')
    return df
     
# convert currency to USD. average HKD-USD rate is 0.1285
# from 2012 Jan to 2023 Mar
def update_currency(df, rate):
     for index, values in df.iterrows():
          unit = str(df.loc[index, "Unit"])
          unit_split = unit.split(" ")
          if len(unit_split) > 1 and unit_split[1] == "'000":
          # print(str(df.loc[index, "Unit"]).split(" ")[1])
               if str(df.loc[index, "Unit"]).split(" ")[1] == "'000":
                    # there is a few columns with commas in the number, remove them
                    df.loc[index, "Value"] = df.loc[index, "Value"].replace(",", "")
                    if str(df.loc[index, "Value"]) == "0@T":
                         df.loc[index, "Value"] = int(0)
                    elif df.loc[index, "Value"] == None:
                         df.loc[index, "Value"] = int(0)
                    elif str(df.loc[index, "Value"]) == '-':
                         df.loc[index, "Value"] = int(0)
                    elif str(df.loc[index, "Value"]).isnumeric():
                         df.loc[index, "Value"] = int(df.loc[index, "Value"])
                         df.loc[index, "Value"] = int(df.loc[index, "Value"] * 1000 * rate)
          elif len(unit_split) > 1 and unit_split[1] == "Mn":
               if str(df.loc[index, "Unit"]).split(" ")[1] == "Mn":
                    # there is a few columns with commas in the number, remove them
                    df.loc[index, "Value"] = df.loc[index, "Value"].replace(",", "")
                    if str(df.loc[index, "Value"]) == "0@Mn":
                         df.loc[index, "Value"] = int(0)
                    elif df.loc[index, "Value"] == None:
                         df.loc[index, "Value"] = int(0)
                    elif str(df.loc[index, "Value"]) == '-':
                         df.loc[index, "Value"] = int(0)
                    elif str(df.loc[index, "Value"]).isnumeric():
                         df.loc[index, "Value"] = int(df.loc[index, "Value"])
                         df.loc[index, "Value"] = int(df.loc[index, "Value"] * 1000000 * rate)
          else:
               df.loc[index, "Value"] = int(0)
     df.rename(columns={"Value": "VALUE_USD"}, inplace=True)
     df.drop(columns=["origin", "Statistics", "Unit"], axis=0, inplace= True)

     return df
# to remove rows with empty values
def remove_empty_values_rows(df):
     df_filtered = df[df['VALUE_USD'] != 0]
     df_filtered['VALUE_USD'] = df_filtered['VALUE_USD'].astype(float)
     return df_filtered

# to concatenate HS code
def concat_hs_code(df, col_name,old_string,new_string):
     df[col_name] = df[col_name].str.replace(old_string, new_string)
     df.rename(columns={col_name: "HS_CODE"}, inplace=True)
     return df

### apply to origin data frame
# call defined function to remove the notes at the bottom of the table
df_origin = remove_notes_at_footer(df_origin)
# call defined function to format date
df_origin = format_date(df_origin)
# # call defined function to add ISO code to dataframe
df_origin = update_iso(df_origin, df_iso, 'Origin')
# # convert currency to USD using defined function
# # this function also clean unse columns
df_origin = update_currency(df_origin, 0.1285)
# # remove empty values rows
df_origin = remove_empty_values_rows(df_origin)
# remove commans and convert datatype to int
# df_origin = clean_comma_in_value(df_origin, "VALUE_USD")


### apply to Consignment data frame
# call defined function to remove the notes at the bottom of the table
df_consign = remove_notes_at_footer(df_consign)
# call defined function to format date
df_consign = format_date(df_consign)
# call defined function to add ISO code to dataframe
df_consign = update_iso(df_consign, df_iso, 'Consignment')
# convert currency to USD using defined function
# this function also clean unse columns
df_consign = update_currency(df_consign, 0.1285)
# remove empty values rows
df_consign = remove_empty_values_rows(df_consign)
# for consignment concat column with hs code and rename column name
df_consign = concat_hs_code(df_consign, 
                            "HKHS 6-digit", 
                            "852351 - SEMICONDUCTOR MEDIA, SOLID-STATE NON-VOLATILE STORAGE DEVICES",
                            "852351")

df_origin.head()                

# note the VALUE_USD column is float
df_origin.to_csv("DB_HK_REEXPORT_ORIGIN_UPDATE_2023_MAR_2023052401.csv", index=False)
df_consign.to_csv("DB_HK_REEXPORT_CONSIGNMENT_UPDATE_2023_MAR_2023052401.csv", index=False)

C:\Users\hboyz\AppData\Local\Temp\ipykernel_21864\1401351955.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['VALUE_USD'] = df_filtered['VALUE_USD'].astype(float)
C:\Users\hboyz\AppData\Local\Temp\ipykernel_21864\1401351955.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['VALUE_USD'] = df_filtered['VALUE_USD'].astype(float)


In [ ]:
df_origin["VALUE_USD"] = df_origin["VALUE_USD"].astype(int) 

In [94]:
# df_origin.sort_values("VALUE_USD", ascending=False)
# type(df_origin.VALUE_USD[0])
# df_origin.VALUE_USD[2]

print(df_origin.dtypes)
df_origin.VALUE_USD.describe()

DATE           object
Trade Type     object
Origin         object
VALUE_USD     float64
ISO            object
dtype: object


count    1.760500e+04
mean     3.189436e+08
std      2.109763e+09
min      1.285000e+05
25%      5.140000e+05
50%      3.212500e+06
75%      2.711350e+07
max      3.195294e+10
Name: VALUE_USD, dtype: float64

In [90]:
df_origin[df_origin["VALUE_USD"] == '1,541']

# 	DATE	Trade Type	Origin	VALUE_USD	ISO
# 20	2012-1-1	Re-exports	BELGIUM	1,541	BEL
# 589	2012-3-1	Re-exports	SWITZERLAND	1,541	CHE

,DATE,Trade Type,Origin,VALUE_USD,ISO


In [80]:
df_origin.shape

(27945, 5)

In [85]:
df_consign[df_consign["ISO"].isna()]

,DATE,Trade Type,Consignment,HS_CODE,VALUE_USD,ISO


In [33]:
df = pd.read_csv("Hk by recorded by reexport.csv")

df["Date"] = 0
for index, value in df.iterrows():
    if df.loc[index, "Month"] == 'Jan':
         df.loc[index, "Date"] = str(df.loc[index, "Year"]) + "-1-1"
    elif df.loc[index, "Month"] == 'Feb':
         df.loc[index, "Date"] = str(df.loc[index, "Year"]) + "-2-1"
    elif df.loc[index, "Month"] == 'Mar':
         df.loc[index, "Date"] = str(df.loc[index, "Year"]) + "-3-1"
    elif df.loc[index, "Month"] == 'Apr':
         df.loc[index, "Date"] = str(df.loc[index, "Year"]) + "-4-1"
    elif df.loc[index, "Month"] == 'May':
         df.loc[index, "Date"] = str(df.loc[index, "Year"]) + "-5-1"
    elif df.loc[index, "Month"] == 'Jun':
         df.loc[index, "Date"] = str(df.loc[index, "Year"]) + "-6-1"
    elif df.loc[index, "Month"] == 'Jul':
         df.loc[index, "Date"] = str(df.loc[index, "Year"]) + "-7-1"
    elif df.loc[index, "Month"] == 'Aug':
         df.loc[index, "Date"] = str(df.loc[index, "Year"]) + "-8-1"
    elif df.loc[index, "Month"] == 'Sep':
         df.loc[index, "Date"] = str(df.loc[index, "Year"]) + "-9-1"
    elif df.loc[index, "Month"] == 'Oct':
         df.loc[index, "Date"] = str(df.loc[index, "Year"]) + "-10-1"
    elif df.loc[index, "Month"] == 'Nov':
         df.loc[index, "Date"] = str(df.loc[index, "Year"]) + "-11-1"
    elif df.loc[index, "Month"] == 'Dec':
         df.loc[index, "Date"] = str(df.loc[index, "Year"]) + "-12-1"
         
    if df.loc[index, "Value"] == '-' or \
        df.loc[index, "Value"] == '0@T' or\
        df.loc[index, "Value"] == None:
         df.loc[index, "Value"] = ""
    else:
        df.loc[index, "Value"] = str(df.loc[index, "Value"])
        
    if df.loc[index, "Unit"] == "HK$ '000":
        df.loc[index, "Unit"] = "HKD"
    else:
        df.loc[index, "Unit"] = ""
df.rename(columns={"Value": "Value_in_thousands"}, inplace=True)
df.drop(columns=["Year","Month", "Statistics"], axis=0, inplace=True)

df.to_csv("Hongkong_Re-export_by_recorded_2023052201.csv", index=False)

In [34]:
df = pd.read_csv("HK by recorded origin.csv")

df["Date"] = 0
for index, value in df.iterrows():
    if df.loc[index, "Month"] == 'Jan':
         df.loc[index, "Date"] = str(df.loc[index, "Year"]) + "-1-1"
    elif df.loc[index, "Month"] == 'Feb':
         df.loc[index, "Date"] = str(df.loc[index, "Year"]) + "-2-1"
    elif df.loc[index, "Month"] == 'Mar':
         df.loc[index, "Date"] = str(df.loc[index, "Year"]) + "-3-1"
    elif df.loc[index, "Month"] == 'Apr':
         df.loc[index, "Date"] = str(df.loc[index, "Year"]) + "-4-1"
    elif df.loc[index, "Month"] == 'May':
         df.loc[index, "Date"] = str(df.loc[index, "Year"]) + "-5-1"
    elif df.loc[index, "Month"] == 'Jun':
         df.loc[index, "Date"] = str(df.loc[index, "Year"]) + "-6-1"
    elif df.loc[index, "Month"] == 'Jul':
         df.loc[index, "Date"] = str(df.loc[index, "Year"]) + "-7-1"
    elif df.loc[index, "Month"] == 'Aug':
         df.loc[index, "Date"] = str(df.loc[index, "Year"]) + "-8-1"
    elif df.loc[index, "Month"] == 'Sep':
         df.loc[index, "Date"] = str(df.loc[index, "Year"]) + "-9-1"
    elif df.loc[index, "Month"] == 'Oct':
         df.loc[index, "Date"] = str(df.loc[index, "Year"]) + "-10-1"
    elif df.loc[index, "Month"] == 'Nov':
         df.loc[index, "Date"] = str(df.loc[index, "Year"]) + "-11-1"
    elif df.loc[index, "Month"] == 'Dec':
         df.loc[index, "Date"] = str(df.loc[index, "Year"]) + "-12-1"
         
    if df.loc[index, "Value"] == '-' or \
        df.loc[index, "Value"] == '0@T' or\
        df.loc[index, "Value"] == None:
         df.loc[index, "Value"] = ""
    else:
        df.loc[index, "Value"] = str(df.loc[index, "Value"])
        
    if df.loc[index, "Unit"] == "HK$ '000":
        df.loc[index, "Unit"] = "HKD"
    else:
        df.loc[index, "Unit"] = ""
        
df.rename(columns={"Value": "Value_in_thousands"}, inplace=True)        
df.drop(columns=["Year","Month", "Statistics"], axis=0, inplace=True)

df.to_csv("Hongkong_Origin_by_recorded_2023052201.csv", index=False)

In [32]:
df = pd.read_csv("HK by unrecorded by origin.csv")

df["Date"] = 0
for index, value in df.iterrows():
    if df.loc[index, "Month"] == 'Jan':
         df.loc[index, "Date"] = str(df.loc[index, "Year"]) + "-1-1"
    elif df.loc[index, "Month"] == 'Feb':
         df.loc[index, "Date"] = str(df.loc[index, "Year"]) + "-2-1"
    elif df.loc[index, "Month"] == 'Mar':
         df.loc[index, "Date"] = str(df.loc[index, "Year"]) + "-3-1"
    elif df.loc[index, "Month"] == 'Apr':
         df.loc[index, "Date"] = str(df.loc[index, "Year"]) + "-4-1"
    elif df.loc[index, "Month"] == 'May':
         df.loc[index, "Date"] = str(df.loc[index, "Year"]) + "-5-1"
    elif df.loc[index, "Month"] == 'Jun':
         df.loc[index, "Date"] = str(df.loc[index, "Year"]) + "-6-1"
    elif df.loc[index, "Month"] == 'Jul':
         df.loc[index, "Date"] = str(df.loc[index, "Year"]) + "-7-1"
    elif df.loc[index, "Month"] == 'Aug':
         df.loc[index, "Date"] = str(df.loc[index, "Year"]) + "-8-1"
    elif df.loc[index, "Month"] == 'Sep':
         df.loc[index, "Date"] = str(df.loc[index, "Year"]) + "-9-1"
    elif df.loc[index, "Month"] == 'Oct':
         df.loc[index, "Date"] = str(df.loc[index, "Year"]) + "-10-1"
    elif df.loc[index, "Month"] == 'Nov':
         df.loc[index, "Date"] = str(df.loc[index, "Year"]) + "-11-1"
    elif df.loc[index, "Month"] == 'Dec':
         df.loc[index, "Date"] = str(df.loc[index, "Year"]) + "-12-1"
         
    if df.loc[index, "Value"] == '-' or \
        df.loc[index, "Value"] == '0@Mn' or\
        df.loc[index, "Value"] == None:
         df.loc[index, "Value"] = ""
         df.loc[index, "Unit"] = ""
    else:
        df.loc[index, "Value"] = str(df.loc[index, "Value"])
        df.loc[index, "Unit"] = "HKD"
     

df.rename(columns={"Value": "Value_in_millions"},inplace=True)
df.drop(columns=["Year","Month", "Statistics"], axis=0, inplace=True)

df.to_csv("Hongkong_Origin_by_unrecorded_2023052201.csv", index=False)

In [35]:
df = pd.read_csv("HK by unrecorded by reexport.csv")

df["Date"] = 0
for index, value in df.iterrows():
    if df.loc[index, "Month"] == 'Jan':
         df.loc[index, "Date"] = str(df.loc[index, "Year"]) + "-1-1"
    elif df.loc[index, "Month"] == 'Feb':
         df.loc[index, "Date"] = str(df.loc[index, "Year"]) + "-2-1"
    elif df.loc[index, "Month"] == 'Mar':
         df.loc[index, "Date"] = str(df.loc[index, "Year"]) + "-3-1"
    elif df.loc[index, "Month"] == 'Apr':
         df.loc[index, "Date"] = str(df.loc[index, "Year"]) + "-4-1"
    elif df.loc[index, "Month"] == 'May':
         df.loc[index, "Date"] = str(df.loc[index, "Year"]) + "-5-1"
    elif df.loc[index, "Month"] == 'Jun':
         df.loc[index, "Date"] = str(df.loc[index, "Year"]) + "-6-1"
    elif df.loc[index, "Month"] == 'Jul':
         df.loc[index, "Date"] = str(df.loc[index, "Year"]) + "-7-1"
    elif df.loc[index, "Month"] == 'Aug':
         df.loc[index, "Date"] = str(df.loc[index, "Year"]) + "-8-1"
    elif df.loc[index, "Month"] == 'Sep':
         df.loc[index, "Date"] = str(df.loc[index, "Year"]) + "-9-1"
    elif df.loc[index, "Month"] == 'Oct':
         df.loc[index, "Date"] = str(df.loc[index, "Year"]) + "-10-1"
    elif df.loc[index, "Month"] == 'Nov':
         df.loc[index, "Date"] = str(df.loc[index, "Year"]) + "-11-1"
    elif df.loc[index, "Month"] == 'Dec':
         df.loc[index, "Date"] = str(df.loc[index, "Year"]) + "-12-1"
         
    if df.loc[index, "Value"] == '-' or \
        df.loc[index, "Value"] == '0@T' or\
        df.loc[index, "Value"] == None:
         df.loc[index, "Value"] = ""
    else:
        df.loc[index, "Value"] = str(df.loc[index, "Value"])
        
    if df.loc[index, "Unit"] == "HK$ '000":
        df.loc[index, "Unit"] = "HKD"
    else:
        df.loc[index, "Unit"] = ""
        
df.rename(columns={"Value": "Value_in_thousands"}, inplace=True)        
df.drop(columns=["Year","Month", "Statistics"], axis=0, inplace=True)

df.to_csv("Hongkong_Re-export_by_unrecorded_2023052201.csv", index=False)